# Generation TEST SET + many test
This notebook allow me to create a 1st testset, based on 1 document. The ouput it's a CSV file.

I made many other different tests inside.

# SETUP

In [4]:
""" I failed to install giskard in poetry environnement, so I had to manualy instal in the Jupyter Notebook"""
%pip install giskard[llm]==2.12.0

  Obtaining dependency information for giskard[llm]==2.12.0 from https://files.pythonhosted.org/packages/54/22/b8a35292c60f09a39c0d8850ff0e87a42b2b7509b35e096c6c4614f635f7/giskard-2.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for cloudpickle>=1.1.1 from https://files.pythonhosted.org/packages/96/43/dae06432d0c4b1dc9e9149ad37b4ca8384cf6eb7700cd9215b177b914f0a/cloudpickle-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for zstandard>=0.10.0 from https://files.pythonhosted.org/packages/c9/79/07f6d2670fa2708ae3b79aabb82da78e9cbdb08d9bafadf8638d356775ff/zstandard-0.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for mlflow-skinny>=2 from https://files.pythonhosted.org/packages/fd/24/8f10ce95f34d16d0573877c1dac9a4fd4b990f26329f7fd739411b82ffc5/mlflow_skinny-2.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for scikit-learn>=1.0 from https://files.pythonhosted.org/packages/8f

In [10]:
%pip install llama-index PyMuPDF

  Obtaining dependency information for llama-index from https://files.pythonhosted.org/packages/7d/d7/a2fb107075e28e8cc2e887a51dbb850a5e75423b0d35cd5cc7b47208238f/llama_index-0.10.37-py3-none-any.whl.metadata
  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/c6/52/1861223ad6ca30c8fae5c70198ca560cbd653b130534e3f4d81064eaa407/PyMuPDF-1.24.4-cp310-none-manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for llama-index-agent-openai<0.3.0,>=0.1.4 from https://files.pythonhosted.org/packages/bb/55/56641aadc604e1e7174467743fb185d730e9eb8ac222f7fbf6df14f38994/llama_index_agent_openai-0.2.5-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-cli<0.2.0,>=0.1.2 from https://files.pythonhosted.org/packages/8c/1b/554b8da1c7b62a7660a3ab0adfdc13a6046cad45a2490c3640728164f058/llama_index_cli-0.1.12-py3-none-any.whl.metadata
  Obtaining dependency information for llama-index-core<0.11.0,>=0.10.35 from https://files.pyt

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(raise_error_if_not_found=True))

# Retrieve and verify the OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise EnvironmentError("OPENAI_API_KEY not defined in the .env file")

# Use the API key
print("OPENAI_API_KEY successfully loaded.")

OPENAI_API_KEY successfully loaded.


In [23]:
import sys

print(sys.executable)
sys.path.append("../RAG_Basic")
from Call_RAG import ask, ask_Llama3
# ask_Llama3("que fais l'ofac ?")

/home/guillaume/.cache/pypoetry/virtualenvs/forestbot-QnTDFiWY-py3.10/bin/python


# Wrap model and dataset with Giskard
Basic RAG with LLAMA3 and GROQ

In [3]:
import giskard
import pandas as pd


def model_predict(df: pd.DataFrame):
    """Wraps the LLM call in a simple Python function.

    The function takes a pandas.DataFrame containing the input variables needed
    by your model, and must return a list of the outputs (one for each row).
    """
    return [ask_Llama3(question) for question in df["question"]]


# Don’t forget to fill the `name` and `description`: they are used by Giskard
# to generate domain-specific tests.
giskard_model = giskard.Model(
    model=model_predict,
    model_type="text_generation",
    name="Climate Change Question Answering",
    description="This model answers question about Forests protection and conservation area in the area of Central Africa",
    feature_names=["question"],
)

INFO:giskard.models.automodel:Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


In [5]:
# Test
# Optional: let’s test that the wrapped model works
examples = [
    "Combien de pays son couvert par l'OFAC ? ",
    "Le Cameroun est-il a jour de ses engagements ?",
]
giskard_dataset = giskard.Dataset(pd.DataFrame({"question": examples}), target=None)

print(giskard_model.predict(giskard_dataset).prediction)

INFO:giskard.datasets.base:Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.


INFO:giskard.datasets.base:Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
INFO:giskard.utils.logging_utils:Predicted dataset with shape (2, 1) executed in 0:00:08.347796


["L'OFAC (Observatoire des Forêts d'Afrique Centrale) couvre 10 pays d'Afrique centrale. Thanks for asking!"
 'The text does not explicitly state whether Cameroon is up to date with its commitments. However, it mentions that Cameroon has produced a National Action Plan to Combat Desertification (PAN/LCD) in 2006 and has implemented several initiatives to combat desertification and land degradation. Thanks for asking!']


# Scan model for vulnerabilities 

In [6]:
report = giskard.scan(giskard_model, giskard_dataset, only="hallucination")

INFO:giskard.scanner.logger:Running detectors: ['LLMImplausibleOutputDetector', 'LLMBasicSycophancyDetector']


🔎 Running scan…
Estimated calls to your model: ~30
Estimated LLM calls for evaluation: 22

Running detector LLMImplausibleOutputDetector…


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:giskard.datasets.base:Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 2

LLMImplausibleOutputDetector: 0 issue detected. (Took 0:03:25.032103)
Running detector LLMBasicSycophancyDetector…


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:giskard.datasets.base:Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to /openai/v1/chat/completions in 2.000000 seconds
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:groq._base_client:Retrying request to

LLMBasicSycophancyDetector: 1 issue detected. (Took 0:10:22.444406)
Scan completed: 1 issue found. (Took 0:13:47.479588)
LLM-assisted detectors have used the following resources:
OpenAI LLM calls for evaluation: 21 (10312 prompt tokens and 1595 sampled tokens)



In [7]:
display(report)

<iframe id="scan-139806139275856" srcdoc="<!doctype html>
<html lang="en">

<head>
 <title>Giskard Scan Results</title>
 <base target="_blank">
 <meta charset="utf-8">
 
<style>pre code.hljs{display:block;overflow-x:auto;padding:1em}code.hljs{padding:3px 5px}
/*!
 Theme: GitHub Dark
 Description: Dark theme as seen on github.com
 Author: github.com
 Maintainer: @Hirse
 Updated: 2021-05-15

 Outdated base version: https://github.com/primer/github-syntax-dark
 Current colors taken from GitHub's CSS
*/.hljs{background:#0d1117;color:#c9d1d9}.hljs-doctag,.hljs-keyword,.hljs-meta .hljs-keyword,.hljs-template-tag,.hljs-template-variable,.hljs-type,.hljs-variable.language_{color:#ff7b72}.hljs-title,.hljs-title.class_,.hljs-title.class_.inherited__,.hljs-title.function_{color:#d2a8ff}.hljs-attr,.hljs-attribute,.hljs-literal,.hljs-meta,.hljs-number,.hljs-operator,.hljs-selector-attr,.hljs-selector-class,.hljs-selector-id,.hljs-variable{color:#79c0ff}.hljs-meta .hljs-string,.hljs-regexp,.hljs-string{color:#a5d6ff}.hljs-built_in,.hljs-symbol{color:#ffa657}.hljs-code,.hljs-comment,.hljs-formula{color:#8b949e}.hljs-name,.hljs-quote,.hljs-selector-pseudo,.hljs-selector-tag{color:#7ee787}.hljs-subst{color:#c9d1d9}.hljs-section{color:#1f6feb;font-weight:700}.hljs-bullet{color:#f2cc60}.hljs-emphasis{color:#c9d1d9;font-style:italic}.hljs-strong{color:#c9d1d9;font-weight:700}.hljs-addition{background-color:#033a16;color:#aff5b4}.hljs-deletion{background-color:#67060c;color:#ffdcd7}.hljs-copy-wrapper{overflow:hidden;position:relative}.hljs-copy-button:focus,.hljs-copy-wrapper:hover .hljs-copy-button{transform:translateX(0)}.hljs-copy-button{background-color:#2d2b57;background-color:var(--hljs-theme-background);background-image:url('data:image/svg+xml;charset=utf-8,<svg xmlns="http://www.w3.org/2000/svg" width="16" height="16" fill="none" viewBox="0 0 24 24"><path fill="%23fff" fill-rule="evenodd" d="M6 5a1 1 0 0 0-1 1v14a1 1 0 0 0 1 1h12a1 1 0 0 0 1-1V6a1 1 0 0 0-1-1h-2a1 1 0 1 1 0-2h2a3 3 0 0 1 3 3v14a3 3 0 0 1-3 3H6a3 3 0 0 1-3-3V6a3 3 0 0 1 3-3h2a1 1 0 0 1 0 2H6Z" clip-rule="evenodd"/><path fill="%23fff" fill-rule="evenodd" d="M7 3a2 2 0 0 1 2-2h6a2 2 0 0 1 2 2v2a2 2 0 0 1-2 2H9a2 2 0 0 1-2-2V3Zm8 0H9v2h6V3Z" clip-rule="evenodd"/></svg>');background-position:50%;background-repeat:no-repeat;border:1px solid #ffffff22;border-radius:.25rem;color:#fff;height:2rem;position:absolute;right:1em;text-indent:-9999px;top:1em;transition:background-color .2s ease,transform .2s ease-out;width:2rem}.hljs-copy-button:hover{border-color:#ffffff44}.hljs-copy-button:active{border-color:#ffffff66}.hljs-copy-button[data-copied=true]{background-image:none;text-indent:0;width:auto}@media (prefers-reduced-motion){.hljs-copy-button{transition:none}}.hljs-copy-alert{clip:rect(0 0 0 0);-webkit-clip-path:inset(50%);clip-path:inset(50%);height:1px;overflow:hidden;position:absolute;white-space:nowrap;width:1px}
/*! tailwindcss v3.3.2 | MIT License | https://tailwindcss.com*/*,:after,:before{border:0 solid #e5e7eb;box-sizing:border-box}:after,:before{--tw-content:""}html{-webkit-text-size-adjust:100%;font-feature-settings:normal;font-family:ui-sans-serif,system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji;font-variation-settings:normal;line-height:1.5;-moz-tab-size:4;-o-tab-size:4;tab-size:4}body{line-height:inherit;margin:0}hr{border-top-width:1px;color:inherit;height:0}abbr:where([title]){-webkit-text-decoration:underline dotted;text-decoration:underline dotted}h1,h2,h3,h4,h5,h6{font-size:inherit;font-weight:inherit}a{color:inherit;text-decoration:inherit}b,strong{font-weight:bolder}code,kbd,pre,samp{font-family:ui-monospace,SFMono-Regular,Menlo,Monaco,Consolas,Liberation Mono,Courier New,monospace;font-size:1em}small{font-size:80%}sub,sup{font-size:75%;line-height:0;position:relative;vertical-align:baseline}sub{bottom:-.25em}sup{top:-.5em}table{border-collaps

# Test of Basic RAG with OpenAI chatGPT3.5
Becarefull : this script doesn't save the data set :-(

In [8]:
def model_predict_BasicRAG(df: pd.DataFrame):
    """Wraps the LLM call in a simple Python function.

    The function takes a pandas.DataFrame containing the input variables needed
    by your model, and must return a list of the outputs (one for each row).
    """
    return [ask(question) for question in df["question"]]


giskard_model_BasicRAG = giskard.Model(
    model=model_predict_BasicRAG,
    model_type="text_generation",
    name="Climate Change Question Answering",
    description="This model answers question about Forests protection and Conservation area in the area of Central Africa",
    feature_names=["question"],
)

INFO:giskard.models.automodel:Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


In [ ]:
scan_results_BasicRAG = giskard.scan(giskard_model_BasicRAG)

In [ ]:
display(scan_results_BasicRAG)
scan_results_BasicRAG.to_html("scan_report.html")

# Creation of the Dataset

In [1]:
import pandas as pd
import warnings

pd.set_option("display.max_colwidth", 400)
warnings.filterwarnings("ignore")

from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()

pdf_files = ["../../pdf/EdAP_2020_EN.pdf", "../../pdf/SOF book-web-rev3d-hires.pdf"]
OFAC_document = loader.load(file_path="../../pdf/EdAP 2020_EN.pdf")
splitter = SentenceSplitter(chunk_size=512)

In [2]:
from giskard.rag import KnowledgeBase, generate_testset, QATestset

text_nodes = splitter(OFAC_document)
knowledge_base_df = pd.DataFrame([node.text for node in text_nodes], columns=["text"])
knowledge_base = KnowledgeBase(knowledge_base_df)

# Record the knowlegde base in CSV format to be reused
knowledge_base_df.to_csv("knowledge_base.csv", index=False)

In [13]:
testset = generate_testset(
    knowledge_base,
    num_questions=120,
    agent_description="A chatbot answering questions about the Forest protection and Conservation Area in Central Africa and Congo Basin",
)

2024-05-22 09:47:43,018 pid:17771 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-05-22 09:48:39,299 pid:17771 MainThread giskard.rag  INFO     Found 3 topics in the knowledge base.


Generating questions:   0%|          | 0/120 [00:00<?, ?it/s]

2024-05-22 10:24:57,855 pid:17771 Thread-32 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track


In [20]:
# testset = QATestset.load("OFAC_testset.jsonl")
# testset.to_pandas().head(5)
testset.to_pandas().to_csv("testset.csv")
testset.save("my_testset.jsonl")

# You can easily load it back
loaded_testset = QATestset.load("my_testset.jsonl")  # renommer le tests

# Test Dataset with BasicRAG

In [24]:
from giskard.rag import evaluate
from giskard.rag.metrics.ragas_metrics import ragas_context_precision
# from Call_RAG import ask,ask_Llama3

reportBasicRAG = evaluate(
    ask,
    testset=testset,  # renommer pour cohérence avec loadtestset
    knowledge_base=knowledge_base,
    metrics=[ragas_context_precision],
)

Asking questions to the agent:   0%|          | 0/120 [00:00<?, ?it/s]

2024-05-22 13:34:48,143 pid:17771 Thread-34 backoff      ERROR    Giving up send_request(...) after 1 tries (posthog.request.APIError: [PostHog]  (408))


correctness evaluation:   0%|          | 0/120 [00:00<?, ?it/s]

RAGAS Context Precision evaluation:   0%|          | 0/120 [00:00<?, ?it/s]

2024-05-22 14:30:46,706 pid:17771 Thread-35 (_track) urllib3.connectionpool WARNING  Retrying (Retry(total=3, connect=None, read=None, redirect=None, status=None)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /track


In [25]:
display(reportBasicRAG.to_html(embed=True))

In [28]:
reportBasicRAG.save("OFAC_report with Basic RAG")